In [101]:
from biobb_cmip.cmip.prepare_structure import prepare_structure
from biobb_common.tools import file_utils as fu

In [102]:
import pytraj as pt

input_topology_filename="biobb_cmip/biobb_cmip/test/data/cmip/egfr_topology.zip"
top_file = fu.unzip_top(zip_file=input_topology_filename)

topology = pt.load_topology(filename=top_file)

# Charges
pt_charges = list(topology.charge)

# Elements
standard_elements = { 'hydrogen': 'H', 'carbon': 'C', 'oxygen': 'O', 'nitrogen': 'N', 'sulfur': 'S', 'sodium': 'Na',
                      'chlorine': 'Cl', 'zinc': 'Zn', 'fluorine': 'F', 'magnesium': 'Mg', 'phosphorus': 'P' }
#pt_atom_types = [standard_elements[atom.element] for atom in topology.atoms]
pt_atom_types = []
atoms = list(topology.atoms)
for a, atom in enumerate(atoms):
    element = standard_elements[atom.element]
    # Adapt hydrogens element to CMIP requirements
    if element == 'H':
        # There should we always only 1 bond
        # If you have the error below you may need to updated the pytraj version or reintsall pytraj
        # ValueError: Buffer dtype mismatch, expected 'int' but got 'long'
        bonded_heavy_atom_index = atom.bonded_indices()[0]
        bonded_heavy_atom = atoms[bonded_heavy_atom_index]
        bonded_heavy_atom_element = standard_elements[bonded_heavy_atom.element]
        # Hydrogens bonded to carbons remain as 'H'
        if bonded_heavy_atom_element == 'C':
            pass
        # Hydrogens bonded to oxygen are renamed as 'HO'
        elif bonded_heavy_atom_element == 'O':
            element = 'HO'
        # Hydrogens bonded to nitrogen or sulfur are renamed as 'HN'
        elif bonded_heavy_atom_element == 'N' or bonded_heavy_atom_element == 'S':
            element = 'HN'
        else:
            raise SystemExit(
                'ERROR: Hydrogen bonded to not supported heavy atom: ' + bonded_heavy_atom_element)
    pt_atom_types.append(element)

In [114]:
import MDAnalysis as mda
#from MDAnalysis.topology.guessers import guess_types
from MDAnalysis.topology.guessers import guess_atom_element
import re
import uuid
from pathlib import Path

def create_unique_file_path(parent_dir= None, extension= None):
    if not parent_dir:
        parent_dir = Path.cwd
    if not extension:
        extension = ''
    while True:
        name = str(uuid.uuid4())+extension
        file_path = Path.joinpath(Path(parent_dir).resolve(), name)
        if not file_path.exists():
            return str(file_path)


input_topology_filename="biobb_cmip/biobb_cmip/test/data/cmip/egfr_topology.zip"
top_file = fu.unzip_top(zip_file=input_topology_filename)

with open(top_file) as tf:
    top_lines = tf.readlines()
top_file = create_unique_file_path(parent_dir=Path(top_file).parent.resolve(), extension='.top')
with open(top_file, 'w') as nt:
    for line in top_lines:
        if re.search(r"\.ff.*\.itp", line):
            continue
        nt.write(line)

u = mda.Universe(top_file, topology_format="ITP")
mda_charges = [round(val, 4) for val in u.atoms.charges]
#mda_atom_types = list(guess_types(u.atoms.names))
mda_atom_types = []
for atom in u.atoms:
    atom_element = guess_atom_element(atom.name)
    if atom_element == 'H':
        bonded_atom_element = guess_atom_element(atom.bonded_atoms[0].name)
        if bonded_atom_element == 'O':
            atom_element = 'HO'
        elif bonded_atom_element in ['N', 'S']:
            atom_element = 'HN'
    mda_atom_types.append(atom_element)

/Users/pau/anaconda3/envs/cmip_test/lib/python3.7/site-packages/MDAnalysis/core/universe.py:160: UserWarning: No coordinate reader found for /Users/pau/projects/biobb_cmip/jupyter_notebook/e4c83dcb-cba1-46c9-8636-9bf53a920726/22e5b8be-7314-4da9-98d1-7eef36ef5118.top. Skipping this file.
  'this file.'.format(filename))


In [115]:
import functools
import math

if functools.reduce(lambda x, y : x and y, map(lambda p, q: math.isclose(p,q, abs_tol=0.0001),pt_charges,mda_charges), True):
    print ("The lists l1 and l2 are the same")
else:
    print ("The lists l1 and l2 are not the same")
print(pt_charges[:10])
print(mda_charges[:10])

The lists l1 and l2 are the same
[0.1414, 0.1997, 0.1997, 0.1997, 0.0962, 0.0889, -0.0597, 0.03, 0.03, 0.03]
[0.1414, 0.1997, 0.1997, 0.1997, 0.0962, 0.0889, -0.0597, 0.03, 0.03, 0.03]


In [110]:
import functools
import math

if functools.reduce(lambda x, y : x and y, map(lambda p, q: p == q,pt_atom_types,mda_atom_types), True):
    print ("The lists l1 and l2 are the same")
else:
    print ("The lists l1 and l2 are not the same")
print(pt_atom_types[:10])
print(mda_atom_types[:10])

The lists l1 and l2 are the same
['N', 'HN', 'HN', 'HN', 'C', 'H', 'C', 'H', 'H', 'H']
['N', 'HN', 'HN', 'HN', 'C', 'H', 'C', 'H', 'H', 'H']
